In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import talib
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from utils import get_model_inputs, seed_everything, load_stock_data
from models import Seq2Seq

device = torch.device("cuda")
# device = torch.device("cpu")

In [ ]:
full_data = pd.read_csv('ASX100.csv')
rnn_num_layers = 1
input_feature_len = 10
hidden_size = 2000

n_back = 20
n_forward = 5
rnn_type = 'gru'

val_split = 0.2
batch_size = 6000 
n_epochs = 90
learning_rate = 0.05 
teacher_forcing_ratio = 1

In [ ]:
seed_everything(1)
    
train_data, test_data, scalers = load_stock_data(full_data, n_back, test_date_split="2019-01-01", TI=True)
all_stocks = train_data.keys()

input_data = {stock: get_model_inputs(stock, train_data, n_back, n_forward) for stock in all_stocks}

x = np.concatenate([input_data[stock][0] for stock in all_stocks], axis=0)
y = np.expand_dims(np.concatenate([input_data[stock][1] for stock in all_stocks], axis=0), -1)

x_train, x_val, y_train, y_val = [torch.from_numpy(a) for a in train_test_split(x, y, test_size=val_split, random_state=42)]


train = torch.utils.data.TensorDataset(x_train, y_train)
train_loader = torch.utils.data.DataLoader(dataset=train, 
                                           batch_size=batch_size, 
                                           shuffle=False,
                                           num_workers = 0)

val = torch.utils.data.TensorDataset(x_val, y_val)
val_loader = torch.utils.data.DataLoader(dataset=val, 
                                           batch_size=batch_size, 
                                           shuffle=False,
                                           num_workers=0)


In [ ]:
x_train.size()

In [ ]:
len(all_stocks)

In [ ]:
model = Seq2Seq(input_feature_len, hidden_size, rnn_num_layers, n_back, attn_dim=None, rnn_type=rnn_type, use_attn=False, device=device)

encoder_optimizer = optim.SGD(model.encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(model.decoder.parameters(), lr=learning_rate)

encoder_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(encoder_optimizer, mode='min', factor=0.9, patience=7, threshold=0.00005)
decoder_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(decoder_optimizer, mode='min', factor=0.9, patience=7, threshold=0.00005)

In [ ]:
%%time 
losses = []
losses_val = []
for epoch in range(1, n_epochs+1):
    for i, (input_tensor, target_tensor) in enumerate(train_loader):
        input_tensor, target_tensor = input_tensor.to(device), target_tensor.to(device)
        loss = model.train_step(input_tensor, target_tensor, encoder_optimizer, decoder_optimizer, teacher_forcing_ratio)

        loss_val = 0
        for j, (val_input_tensor, val_target_tensor) in enumerate(val_loader):
            val_input_tensor, val_target_tensor = val_input_tensor.to(device), val_target_tensor.to(device)
            loss_val += model.validate(val_input_tensor, val_target_tensor, n_forward).item()
        loss_val /= len(val_loader)
        
        losses.append(loss)    
        losses_val.append(loss_val)  
        encoder_scheduler.step(loss_val)
        decoder_scheduler.step(loss_val) 
        print(".", end='')
        if epoch % 1 == 0 and (i == len(train_loader)-1):# or i % 10 == 0):
            print('Epoch %d [%d/%d] loss: %.4f loss_val: %.4f lr: %.10f' % (epoch, i, len(train_loader)-1, loss, loss_val, encoder_optimizer.param_groups[0]['lr']))
#     if epoch % 5 == 0:
#         print(model.get_metrics(all_stocks, test_data, scalers, n_back, n_forward).mean())
#         model.save_model("seq2seq_%d_2000_4000_005.pt"%epoch)

In [ ]:
%%time 
metrics = model.get_metrics(all_stocks, test_data, scalers, n_back, n_forward)

In [ ]:
metrics.mean()

In [ ]:
model.save_model("./model_checkpoints/seq2seq_90_2000_6000_005.pt")